## LLM 모델 설정

In [17]:
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field

load_dotenv()

llm = ChatOpenAI(
    model_name="gpt-4o-mini",
    temperature=0.7,
    max_tokens=150,
)

# from langchain.schema import HumanMessage

# response_mini = gpt4o_mini.invoke([HumanMessage(content="Hello, Can you tell me who you are?")])
# print(response_mini.content)

In [18]:
email_conversation = """From: 김철수 (chulsoo.kim@bikecorporation.me)
To: 이은채 (eunchae@teddyinternational.me)
Subject: "ZENESIS" 자전거 유통 협력 및 미팅 일정 제안

안녕하세요, 이은채 대리님,

저는 바이크코퍼레이션의 김철수 상무입니다. 최근 보도자료를 통해 귀사의 신규 자전거 "ZENESIS"에 대해 알게 되었습니다. 바이크코퍼레이션은 자전거 제조 및 유통 분야에서 혁신과 품질을 선도하는 기업으로, 이 분야에서의 장기적인 경험과 전문성을 가지고 있습니다.

ZENESIS 모델에 대한 상세한 브로슈어를 요청드립니다. 특히 기술 사양, 배터리 성능, 그리고 디자인 측면에 대한 정보가 필요합니다. 이를 통해 저희가 제안할 유통 전략과 마케팅 계획을 보다 구체화할 수 있을 것입니다.

또한, 협력 가능성을 더 깊이 논의하기 위해 다음 주 화요일(1월 15일) 오전 10시에 미팅을 제안합니다. 귀사 사무실에서 만나 이야기를 나눌 수 있을까요?

감사합니다.

김철수
상무이사
바이크코퍼레이션
"""

## PydanticOutputParser

In [19]:
class EmailSummary(BaseModel):
    person: str = Field(description="메일을 보낸 사람")
    email: str = Field(description="메일을 보낸 사람의 이메일 주소")
    subject: str = Field(description="메일 제목")
    summary: str = Field(description="메일 본문을 요약한 텍스트")
    date: str = Field(description="메일 본문에 언급된 미팅 날짜와 시간")

# PydanticOutputParser 생성
parser = PydanticOutputParser(pydantic_object=EmailSummary)
print(parser.get_format_instructions())

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"person": {"description": "메일을 보낸 사람", "title": "Person", "type": "string"}, "email": {"description": "메일을 보낸 사람의 이메일 주소", "title": "Email", "type": "string"}, "subject": {"description": "메일 제목", "title": "Subject", "type": "string"}, "summary": {"description": "메일 본문을 요약한 텍스트", "title": "Summary", "type": "string"}, "date": {"description": "메일 본문에 언급된 미팅 날짜와 시간", "title": "Date", "type": "string"}}, "required": ["person", "email", "subject", "summary", "date"]}
```


In [20]:
from langchain_core.prompts import PromptTemplate

prompt = PromptTemplate.from_template(
    """
You are a helpful assistant. Please answer the following questions in KOREAN.

QUESTION:
{question}

EMAIL CONVERSATION:
{email_conversation}

FORMAT:
{format}
"""
)

In [21]:
# format 에 PydanticOutputParser의 부분 포맷팅(partial) 추가
prompt = prompt.partial(format=parser.get_format_instructions())
chain = prompt | llm
response = chain.invoke(
    {
        "email_conversation": email_conversation,
        "question": "이메일 내용중 주요 내용을 추출해 주세요.",
    }
)

In [28]:
response.model_dump()

{'content': '```json\n{\n  "person": "김철수",\n  "email": "chulsoo.kim@bikecorporation.me",\n  "subject": "\\"ZENESIS\\" 자전거 유통 협력 및 미팅 일정 제안",\n  "summary": "김철수 상무는 이은채 대리에게 바이크코퍼레이션의 자전거 유통 협력을 제안하며, ZENESIS 모델에 대한 브로슈어 요청과 미팅 일정을 제안하였다.",\n  "date": "1월 15일 오전 10시"\n}\n```',
 'additional_kwargs': {'refusal': None},
 'response_metadata': {'token_usage': {'completion_tokens': 123,
   'prompt_tokens': 601,
   'total_tokens': 724,
   'completion_tokens_details': {'accepted_prediction_tokens': 0,
    'audio_tokens': 0,
    'reasoning_tokens': 0,
    'rejected_prediction_tokens': 0},
   'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}},
  'model_name': 'gpt-4o-mini-2024-07-18',
  'system_fingerprint': 'fp_34a54ae93c',
  'id': 'chatcmpl-BdAmGSCTAjw1t9w4KWxswT5H4UJOU',
  'finish_reason': 'stop',
  'logprobs': None},
 'type': 'ai',
 'name': None,
 'id': 'run-b799c632-c3d9-4f31-93e0-519a414cad31-0',
 'example': False,
 'tool_calls': [],
 'invalid_tool_calls': [],
 'usage_metada

In [ ]:
print(response.content)

```json
{
  "person": "김철수",
  "email": "chulsoo.kim@bikecorporation.me",
  "subject": "\"ZENESIS\" 자전거 유통 협력 및 미팅 일정 제안",
  "summary": "김철수 상무는 이은채 대리에게 바이크코퍼레이션의 자전거 유통 협력을 제안하며, ZENESIS 모델에 대한 브로슈어 요청과 미팅 일정을 제안하였다.",
  "date": "1월 15일 오전 10시"
}
```


In [32]:
structured_output = parser.parse(response.content)
structured_output.model_dump()

{'person': '김철수',
 'email': 'chulsoo.kim@bikecorporation.me',
 'subject': '"ZENESIS" 자전거 유통 협력 및 미팅 일정 제안',
 'summary': '김철수 상무는 이은채 대리에게 바이크코퍼레이션의 자전거 유통 협력을 제안하며, ZENESIS 모델에 대한 브로슈어 요청과 미팅 일정을 제안하였다.',
 'date': '1월 15일 오전 10시'}

In [ ]:
structured_output.model_dump()

dict

In [34]:
chain2 = prompt | llm | parser

response2 = chain2.invoke({
        "email_conversation": email_conversation,
        "question": "이메일 내용중 주요 내용을 추출해 주세요.",
    }
)


In [37]:
response2

EmailSummary(person='김철수', email='chulsoo.kim@bikecorporation.me', subject='"ZENESIS" 자전거 유통 협력 및 미팅 일정 제안', summary='바이크코퍼레이션의 김철수가 ZENESIS 자전거에 대한 브로슈어 요청과 협력 논의를 위한 미팅 제안을 보냈습니다.', date='1월 15일 오전 10시')

In [36]:
response2.model_dump()

{'person': '김철수',
 'email': 'chulsoo.kim@bikecorporation.me',
 'subject': '"ZENESIS" 자전거 유통 협력 및 미팅 일정 제안',
 'summary': '바이크코퍼레이션의 김철수가 ZENESIS 자전거에 대한 브로슈어 요청과 협력 논의를 위한 미팅 제안을 보냈습니다.',
 'date': '1월 15일 오전 10시'}

### with_structured_output

In [39]:
llm_with_structured = ChatOpenAI(
    model_name="gpt-4o-mini",
    temperature=0.7,
    max_tokens=150,
).with_structured_output(EmailSummary)

In [40]:
answer = llm_with_structured.invoke(email_conversation)
answer

EmailSummary(person='이은채', email='eunchae@teddyinternational.me', subject='"ZENESIS" 자전거 유통 협력 및 미팅 일정 제안', summary='김철수 상무가 이은채 대리에게 바이크코퍼레이션의 자전거 "ZENESIS" 유통 협력에 대해 문의하며, 상세 브로슈어 요청과 함께 1월 15일 오전 10시에 미팅을 제안했다.', date='2023-01-12')

In [41]:
answer.model_dump()

{'person': '이은채',
 'email': 'eunchae@teddyinternational.me',
 'subject': '"ZENESIS" 자전거 유통 협력 및 미팅 일정 제안',
 'summary': '김철수 상무가 이은채 대리에게 바이크코퍼레이션의 자전거 "ZENESIS" 유통 협력에 대해 문의하며, 상세 브로슈어 요청과 함께 1월 15일 오전 10시에 미팅을 제안했다.',
 'date': '2023-01-12'}

In [42]:
chain3 = prompt | llm_with_structured

response3 = chain3.invoke({
        "email_conversation": email_conversation,
        "question": "이메일 내용중 주요 내용을 추출해 주세요.",
    }
)


In [43]:
response3.model_dump()

{'person': '김철수',
 'email': 'chulsoo.kim@bikecorporation.me',
 'subject': '"ZENESIS" 자전거 유통 협력 및 미팅 일정 제안',
 'summary': '김철수 상무가 이은채 대리에게 ZENESIS 자전거에 대한 브로슈어 요청 및 협력 논의를 위한 미팅 제안(1월 15일 오전 10시)을 보냈습니다.',
 'date': '2024-01-15T10:00:00'}

## JsonOutputParser

In [2]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from pydantic import BaseModel, Field
from langchain_openai import ChatOpenAI

from dotenv import load_dotenv
load_dotenv()

llm = ChatOpenAI(
    model_name="gpt-4o-mini",
    temperature=0.7,
    max_tokens=150,
)

In [3]:
class Topic(BaseModel):
    description: str = Field(description="주제에 대한 간결한 설명")
    hashtags: str = Field(description="해시태그 형식의 키워드(2개 이상)")

In [ ]:
parser = JsonOutputParser(pydantic_object=Topic)
print(parser.get_format_instructions())

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "당신은 친절한 AI 어시스턴트 입니다. 질문에 간결하게 답변하세요."),
        ("user", "#Format: {format_instructions}\n\n#Question: {question}"),
    ]
)
prompt = prompt.partial(format_instructions=parser.get_format_instructions())
chain = prompt | llm | parser

question = "지구 온난화의 심각성 대해 알려주세요."
answer = chain.invoke({"question": question})

In [9]:
parser = JsonOutputParser(pydantic_object=Topic)
print(parser.get_format_instructions())

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"description": {"description": "주제에 대한 간결한 설명", "title": "Description", "type": "string"}, "hashtags": {"description": "해시태그 형식의 키워드(2개 이상)", "title": "Hashtags", "type": "string"}}, "required": ["description", "hashtags"]}
```


In [5]:
answer

{'description': '지구 온난화는 기후 변화로 인해 지구의 평균 온도가 상승하고 있는 현상으로, 생태계와 인류에 심각한 영향을 미치고 있습니다.',
 'hashtags': '#지구온난화 #기후변화'}

In [6]:
type(answer)

dict

### without Pydantic

In [10]:
# JSON 출력 파서 초기화
parser = JsonOutputParser()

# 프롬프트 템플릿을 설정합니다.
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "당신은 친절한 AI 어시스턴트 입니다. 질문에 간결하게 답변하세요."),
        ("user", "#Format: {format_instructions}\n\n#Question: {question}"),
    ]
)
prompt = prompt.partial(format_instructions=parser.get_format_instructions())
chain = prompt | llm | parser

question = "지구 온난화에 대해 알려주세요. 온난화에 대한 설명은 `description`에, 관련 키워드는 `hashtags`에 담아주세요."
response = chain.invoke({"question": question})

print(response)

{'description': '지구 온난화는 지구의 평균 기온이 상승하는 현상으로, 주로 온실가스의 증가로 인해 발생합니다. 이로 인해 기후 변화, 해수면 상승, 생태계 파괴 등의 문제가 발생할 수 있습니다.', 'hashtags': ['#지구온난화', '#기후변화', '#온실가스', '#환경문제', '#지구']}


In [8]:
parser.get_format_instructions()

'Return a JSON object.'

## StructuredOutputParser

In [11]:
from langchain.output_parsers import ResponseSchema, StructuredOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

response_schemas = [
    ResponseSchema(name="description", description="사용자의 질문에 대한 답변"),
    ResponseSchema(
        name="hashtag",
        description="답변 내 주요 키워드",
    ),
]

In [19]:
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)
format_instructions = output_parser.get_format_instructions(only_json=True)
print(format_instructions)

# prompt = PromptTemplate(
#     template = """당신은 친절한 AI 어시스턴트 입니다. 질문에 간결하게 답변하세요.\n#Format: {format_instructions}\n#question: {question}""",
#     # input_variables=["question"],
#     # partial_variables={"format_instructions": format_instructions}
# )

prompt = PromptTemplate.from_template(
    template = """당신은 친절한 AI 어시스턴트 입니다. 질문에 간결하게 답변하세요.\n#Format: {format_instructions}\n#question: {question}""",
)
prompt = prompt.partial(format_instructions=output_parser.get_format_instructions())
chain = prompt | llm | output_parser

question = "지구 온난화의 심각성 대해 알려주세요."
answer = chain.invoke({"question": question})


```json
{
	"description": string  // 사용자의 질문에 대한 답변
	"hashtag": string  // 답변 내 주요 키워드
}
```


In [20]:
answer

{'description': '지구 온난화는 기후 변화로 인해 지구의 평균 기온이 상승하는 현상으로, 해수면 상승, 극단적인 날씨, 생태계 파괴 등 다양한 문제를 초래합니다. 이는 인류와 자연에 심각한 영향을 미치고 있어 빠른 대응이 필요합니다.',
 'hashtag': '#지구온난화 #기후변화 #환경문제'}

In [3]:
from langgraph.graph import StateGraph, START, END
from typing import TypedDict

# 그래프의 상태를 정의하는 클래스
class MyState(TypedDict):
    counter: int

graph = StateGraph(MyState)

def increment(state):
    return {"counter": state["counter"] + 1}

graph.add_node("increment", increment)

graph.add_edge(START, "increment")

graph.add_edge("increment", END)

app = graph.compile()

# 초기 상태로 0을 제공
result = app.invoke({"counter": 5})
print(result)

{'counter': 6}
